In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list
!kaggle datasets download -d alessandrasala79/ai-vs-human-generated-dataset
!pip install timm

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
Dataset URL: https://www.kaggle.com/datasets/alessandrasala79/ai-vs-human-generated-dataset
License(s): apache-2.0
100% 9.74G/9.76G [00:44<00:00, 221MB/s]
100% 9.76G/9.76G [00:44<00:00, 237MB/s]


In [ ]:
!unzip /content/ai-vs-human-generated-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_data/efd938a800124cf69ec656cd2d518fa5.jpg  
  inflating: train_data/efd97dc6cf2d44b4b635b20415c516c1.jpg  
  inflating: train_data/efd9d70912ee498485cabe62c2a9943e.jpg  
  inflating: train_data/efd9facc4ca04695ad2710f43a33219d.jpg  
  inflating: train_data/efda2fdd29ae4c19ac1ecf6c1a83166b.jpg  
  inflating: train_data/efdc29bae2ba4847aa23f5549ce3e89b.jpg  
  inflating: train_data/efdcd2eab54d4c6f87b6cb25dbaf139a.jpg  
  inflating: train_data/efdceffbac1e445e987ac4e06d8775ec.jpg  
  inflating: train_data/efde74f673c341e09ab5184fe29759da.jpg  
  inflating: train_data/efdf66dfcf4d4e53b7fc9b55cd563d0d.jpg  
  inflating: train_data/efdfd51607ed4ff8ad14ffeeb798f977.jpg  
  inflating: train_data/efe1570296ed40e39d5fc5bb77105920.jpg  
  inflating: train_data/efe241b299c0449e80982b5076920fe2.jpg  
  inflating: train_data/efe372a5186d47b8b17d04c913bbc071.jpg  
  inflating: train_data/efe4a3f0ec094c1798d277260e26ce5a.jpg  
  in

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import timm
import albumentations as albu
from albumentations.pytorch import ToTensorV2

from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import AdamW, Adam
from torch.optim.lr_scheduler import StepLR

from sklearn.metrics import f1_score
from tqdm import tqdm
import copy

import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
import albumentations as albu
from albumentations.pytorch import ToTensorV2

from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.1 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
class CustomDataset(Dataset):
    """
    A custom Dataset for images labeled as either real or AI-generated.

    Args:
        df (pd.DataFrame): DataFrame with columns ['file_name', 'label'].
        data_dir (str): Directory where the image files are stored.
        transform (albu.Compose, optional): Albumentations transform to apply.

    Returns:
        (image, label): Transformed image (as a torch.Tensor) and the corresponding label.
    """
    def __init__(self, df, data_dir, transform=None):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transform

        # Pre-extract image paths and labels for speed
        self.image_paths = [
            os.path.join(self.data_dir, fname) for fname in df['file_name'].values
        ]
        self.labels = df['label'].values

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 1. Load image
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        image = np.array(image)  # Convert PIL image to NumPy

        # 2. Apply transforms
        if self.transform is not None:
            image = self.transform(image=image)['image']

        # 3. Fetch label
        label = self.labels[idx]

        return image, label


In [ ]:
BATCH_SIZE = 32
SEED = 42
IMG_SIZE = 224

# ======================
# 3. Read CSV + Split
# ======================
train_csv_path = "/content/train.csv"
train_df = pd.read_csv(train_csv_path)

# Stratified train/val split
train_df, val_df = train_test_split(
    train_df,
    test_size=0.3,
    random_state=SEED,
    stratify=train_df['label']
)

val_df, test_df =  train_test_split(
    val_df,
    test_size=0.5,
    random_state=SEED,
    stratify=val_df['label']
)

# ======================
# 4. Define Transforms
# ======================
train_transform = albu.Compose([
    albu.HorizontalFlip(p=0.5),
    albu.Resize(IMG_SIZE, IMG_SIZE),
    albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = albu.Compose([
    albu.Resize(IMG_SIZE, IMG_SIZE),
    albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = albu.Compose([
    albu.Resize(IMG_SIZE, IMG_SIZE),
    albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


# ======================
# 5. Create Datasets & Dataloaders
# ======================
train_data_dir = "/content/"

train_dataset = CustomDataset(
    df=train_df,
    data_dir=train_data_dir,
    transform=train_transform
)

val_dataset = CustomDataset(
    df=val_df,
    data_dir=train_data_dir,
    transform=val_transform
)

test_dataset = CustomDataset(
    df=val_df,
    data_dir=train_data_dir,
    transform=test_transform
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4
)

# ======================
# 6. Quick Sanity Check
# ======================
print("Train Batch:")
for images, labels in train_dataloader:
    print(" Images shape:", images.shape)
    print(" Labels:", labels)
    break

print("\nValidation Batch:")
for images, labels in val_dataloader:
    print(" Images shape:", images.shape)
    print(" Labels:", labels)
    break

print("\Test Batch:")
for images, labels in test_dataloader:
    print(" Images shape:", images.shape)
    print(" Labels:", labels)
    break

Train Batch:
 Images shape: torch.Size([32, 3, 224, 224])
 Labels: tensor([0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 1, 1, 0])

Validation Batch:
 Images shape: torch.Size([32, 3, 224, 224])
 Labels: tensor([0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 1, 0])
\Test Batch:
 Images shape: torch.Size([32, 3, 224, 224])
 Labels: tensor([0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 1, 0])


In [ ]:
import timm

class ViTClassifier(nn.Module):
    def __init__(self, model_name="vit_base_patch16_224", num_classes=2, pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.head.in_features
        self.model.head = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

model_name = "vit_base_patch16_224"
num_classes = 2

model = ViTClassifier(model_name=model_name, num_classes=num_classes, pretrained=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Model loaded on:", device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Model loaded on: cuda


In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate_one_epoch(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def test_one_epoch(model, loader, criterion, device):

    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Testing", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [ ]:
EPOCHS = 20
best_val_acc = 0.0
patience = 4
wait = 0


for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    train_loss, train_acc = train_one_epoch(model, train_dataloader, criterion, optimizer, device)

    val_loss, val_acc = validate_one_epoch(model, val_dataloader, criterion, device)

    scheduler.step()

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), f"VIT-{val_acc}.pth")
        print(" --> Best model saved!")
        wait = 0
    else:
        wait += 1
        print(f" --> No improvement. Patience counter: {wait}/{patience}")

        if wait >= patience:
            print("Early stopping triggered! Training stopped.")
            break

    if (epoch + 1) % 5 == 0:
        test_loss, test_acc = test_one_epoch(model, test_dataloader, criterion, device)
        print(f" [Test @ Epoch {epoch+1}] Loss: {test_loss:.4f} | Acc: {test_acc:.4f}")


Epoch 1/20


Training:   0%|          | 0/1749 [00:00<?, ?it/s]

Validating:   0%|          | 0/375 [00:00<?, ?it/s]

Train Loss: 0.1200 | Train Acc: 0.9540
Val Loss:   0.0353 | Val Acc:   0.9877
 --> Best model saved!

Epoch 2/20


Training:   0%|          | 0/1749 [00:00<?, ?it/s]

...

Testing:   0%|          | 0/375 [00:00<?, ?it/s]

 [Test @ Epoch 15] Loss: 0.0240 | Acc: 0.9941

Epoch 16/20


Training:   0%|          | 0/1749 [00:00<?, ?it/s]

Validating:   0%|          | 0/375 [00:00<?, ?it/s]

Train Loss: 0.0032 | Train Acc: 0.9990
Val Loss:   0.0230 | Val Acc:   0.9935
 --> No improvement. Patience counter: 4/4
Early stopping triggered! Training stopped.


In [ ]:
test_csv_path = "/content/test.csv"
test_data_dir = "/content/"

test_df = pd.read_csv(test_csv_path)

print("Test DataFrame head:\n", test_df.head())

test_transform = albu.Compose([
    albu.Resize(IMG_SIZE, IMG_SIZE),
    albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

class TestDataset(Dataset):
    """
    A custom Dataset for test images (with no or unknown labels).
    """

    def __init__(self, df, data_dir, transform=None):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transform

        # We'll assume there's a 'file_name' column in test_df
        self.image_paths = [
            os.path.join(self.data_dir, fname) for fname in df["id"].values
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        image = np.array(image)

        if self.transform is not None:
            image = self.transform(image=image)['image']

        return image

# Create the test dataset
test_dataset = TestDataset(test_df, test_data_dir, transform=test_transform)

# Create the test dataloader
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f"\nNumber of test samples: {len(test_dataset)}")

# Quick check
for images in test_loader:
    print("Test Batch - Images shape:", images.shape)
    break

Test DataFrame head:
                                                   id
0  test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg
1  test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg
2  test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg
3  test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg
4  test_data_v2/a16495c578b7494683805484ca27cf9f.jpg

Number of test samples: 5540
Test Batch - Images shape: torch.Size([32, 3, 224, 224])


In [ ]:
best_model_path = f"VIT-{best_val_acc}.pth"
model.load_state_dict(torch.load(best_model_path, map_location=device))

model.eval()
print(f"Loaded best model weights from: {best_model_path}")
model.eval()

all_predictions = []

with torch.no_grad():
    for images in tqdm(test_loader, desc="Predicting on Test Set"):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_predictions.extend(preds.cpu().numpy())


<ipython-input-11-554b2c2fc9ce>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path, map_location=device))


FileNotFoundError: [Errno 2] No such file or directory: 'best_vit_model.pth'

In [ ]:
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "label": all_predictions
})

submission_df

In [ ]:
# @title label

from matplotlib import pyplot as plt
submission_df['label'].plot(kind='hist', bins=20, title='label')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
submission_df.to_csv('submission.csv', index=False)